In [135]:
import glob

# Reads in all the files and concatenates them into one
read_files = glob.glob("CEK60kCorpus_third_20060330/cek*.txt")
with open("data/CEK60kCorpus_combined.txt", "wb") as outfile:
    for f in read_files:
        with open(f, "rb") as infile:
            outfile.write(infile.read())

In [1]:
# Reads in files
file = open('data/CEK60kCorpus_combined.txt', 'r', encoding='utf-8') 
page = file.read() 
file.close()
page = page.split('\n') # Creates list of sentences

In [2]:
import re

numbers = []
ch_sentences = []
eng_sentences = []
kor_sentences = []

ch_word_to_sentencenum_dict = {}
num_CEK60k_sentence_dict = {}

def AddWordToDict(word_list, sentence_num, input_dict):
    exclude = {'.':0, '':0,',':0}
    for x in word_list:
        if x not in exclude:
            if input_dict.get(x) != None:
                input_dict[x] += [sentence_num]
            else:
                input_dict[x] = [sentence_num]

for i,x in enumerate(page):
    if re.search('^[0-9]{1,5}:',x) != None:
        # Chinese sentence
        #chn = page[i].encode('l1').decode('gb2312') # For decoding mojibake
        chn = page[i]
        #print(chn)
        
        chn_split = chn.split(':')

        chn_split[0] = chn_split[0].replace('\ufeff','')
        sentence_number = int(chn_split[0])
        chn = chn_split[1] # Removes preceding number
                
        chn_segmented = chn.split("\\") # List of segmented Chinese words
        chn_segmented = list(set(chn_segmented))
        AddWordToDict(chn_segmented, sentence_number, ch_word_to_sentencenum_dict) # Adds words to dictionary
        
        chn = chn.replace("\\", "") # Removes backslashes to produce cleaned sentence
        
        # English sentence
        eng = page[i+1]
        eng = eng.replace("#",'')
        
        # Korean sentence
        #kor = page[i+2].encode('l1').decode('EUC-KR') # For decoding mojibake
        kor = page[i+2]
        kor = kor.replace("#",'')
        
        # Appends numbers and sentences to lists
        numbers.append(sentence_number)
        ch_sentences.append(chn)
        eng_sentences.append(eng)
        kor_sentences.append(kor)
        
        # Adds sentences to dictionary
        num_CEK60k_sentence_dict[sentence_number] = {'chn':chn, 'eng':eng ,'kor':kor}
        #print(chn,eng,kor)
        
        
print(len(numbers))
print(len(ch_sentences))
print(len(eng_sentences))
print(len(kor_sentences))

61853
61853
61853
61853


In [3]:
# Creates a Pandas dataframe from data
import pandas as pd
df = pd.DataFrame({'id':numbers, 'Chinese':ch_sentences,'English':eng_sentences,'Korean':kor_sentences})
df.sort_values(by=['id'])

df.to_csv('data/Ch_En_Kor_sentences.csv',index=False)

In [4]:
# Saves dictionaries to pickle
import pickle

with open('data/ch_word_to_sentencenum_dict.pkl', 'wb') as handle:
    pickle.dump(ch_word_to_sentencenum_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('data/num_CEK60k_sentence_dict.pkl', 'wb') as handle:
    pickle.dump(num_CEK60k_sentence_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)